# Imports

In [1]:
import numpy as np
import os
import pytest
from types import SimpleNamespace
import ipywidgets as widgets
import pyvista as pv
import matplotlib.pyplot as plt
from ipyfilechooser import FileChooser
import meshio

from library.pysolver.solver import *
from library.zoomy_core.model.model import *
import library.zoomy_core.model.initial_conditions as IC
import library.zoomy_core.model.boundary_conditions as BC
from library.pysolver.ode import RK1
import library.zoomy_core.misc.io as io
from library.pysolver.reconstruction import GradientMesh
import library.postprocessing.postprocessing as postprocessing

In [4]:

    level = 2
    settings = Settings(
        name="ShallowMoments2d",
        momentum_eqns=[1, 2] + [3 + l for l in range(2 * level)],
        parameters={"g": 1.0, "C": 16.0, "nu": 0.1},
        reconstruction=recon.constant,
        num_flux=flux.LLF(),
        compute_dt=timestepping.adaptive(CFL=0.45),
        time_end=1.0,
        output_snapshots=30,
        output_dir='output_test'
    )


In [5]:

    main_dir = os.getenv("ZOOMY_DIR")
    mesh = Mesh.load_gmsh(
        os.path.join(main_dir, "meshes/{}_2d/mesh_coarse.msh".format('triangle')),
        'triangle',
    )


In [6]:

    bc_tags = ["left", "right", "top", "bottom"]
    bc_tags_periodic_to = ["right", "left", "bottom", "top"]

    bcs = BC.BoundaryConditions(
        [
            BC.Periodic(tag=tag, periodic_to_physical_tag=tag_periodic_to)
            for (tag, tag_periodic_to) in zip(bc_tags, bc_tags_periodic_to)
        ]
    )

In [7]:

    ic = IC.RP(
        left=lambda n_field: np.array(
            [2.0, 0.0, 0.0] + [0.0 for l in range(2 * level)]
        ),
        right=lambda n_field: np.array(
            [1.0, 0.0, 0.0] + [0.0 for l in range(2 * level)]
        ),
    )

In [8]:


    model = ShallowMoments2d(
        dimension=2,
        fields=3 + 2 * level,
        aux_variables=0,
        parameters=settings.parameters,
        boundary_conditions=bcs,
        initial_conditions=ic,
        settings={"friction": ["chezy"]},
    )

In [9]:

    fvm_unsteady_semidiscrete(mesh, model, settings, RK1)

ShallowMoments2d: Progress: 100% ██████████████████████████████████████████████████ 


Settings(name='ShallowMoments2d', momentum_eqns=[1, 2, 3, 4, 5, 6], parameters={'g': 1.0, 'C': 16.0, 'nu': 0.1}, reconstruction=<function constant at 0x7f05477fc670>, reconstruction_edge=<function constant_edge at 0x7f05477fc700>, num_flux=<function LLF.<locals>.flux at 0x7f05946c2700>, nc_flux=<function segmentpath.<locals>.nc_flux at 0x7f0547807820>, compute_dt=<function adaptive.<locals>.compute_dt at 0x7f059880bca0>, time_end=1.0, truncate_last_time_step=True, output_snapshots=30, output_write_all=False, output_dir='output_test', output_clean_dir=True)

In [10]:

    mesh = GradientMesh.fromMesh(mesh)

    io.generate_vtk(settings.output.directory)
    # postprocessing.recover_3d_from_smm_as_vtk(
    #     model,
    #     settings.output.directory,
    #     os.path.join(settings.output.directory, "mesh.hdf5"),
    #     os.path.join(settings.output.directory, "fields.hdf5"),
    #     Nz=10,
    #     start_at_time=1.0,
    # )

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

...

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

In [11]:
accordion = widgets.Accordion(children=[widgets.IntSlider(), widgets.Text()], titles=('Slider', 'Text'))
tab_nest = widgets.Tab()
tab_nest.children = [accordion, accordion]
tab_nest.titles = ('An accordion', 'Copy of the accordion')


In [12]:
accordion

Accordion(children=(IntSlider(value=0), Text(value='')), titles=('Slider', 'Text'))

In [13]:
tab_nest

In [2]:
main_dir = os.getenv("ZOOMY_DIR")
default_mesh_path = os.path.join(main_dir, "meshes/quad_2d")
# Create and display a FileChooser widget
fc = FileChooser(default_mesh_path)
fc.default_path = default_mesh_path
fc.default_filename = 'mesh_coarse.msh'
fc.filter_pattern = '*.msh'
fc.reset()
display(fc)


FileChooser(path='/home/ingo/Git/SMM/shallow-moments-simulation/meshes/quad_2d', filename='mesh_coarse.msh', t…

In [ ]:
# load mesh with pyvista
# extract physical tags from mesh (loadmesh association table has that I think)
# but I do not want to load the whole mesh yet...
# mesh stats?
# combine in one tab
# callbacks, such that chaning the value automatically triggers a recalculation


In [6]:

filename = fc.selected
mesh = Mesh.load_gmsh(filename, 'quad')
mesh.write_to_file_vtk('.tmp/mesh.vtk')


Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

In [12]:
[tag.decode('UTF-8') for tag in mesh.boundary_tag_names]

['bottom', 'left', 'right', 'top']

In [5]:

reader = pv.get_reader('.tmp/mesh.vtk')
mesh = reader.read()
plotter = pv.Plotter()
plotter.add_mesh(mesh, color=True, show_edges=True)
plotter.link_views()
plotter.show(screenshot='U.png')


/home/ingo/.local/lib/python3.9/site-packages/pyvista/jupyter/notebook.py:34: UserWarning: Failed to use notebook backend: 

cannot import name 'vtk' from 'trame.widgets' (/home/ingo/.local/lib/python3.9/site-packages/trame/widgets/__init__.py)

Falling back to a static output.
  warnings.warn(
